## Flood Prediction in Malawi
https://zindi.africa/competitions/2030-vision-flood-prediction-in-malawi

**Problem Statement:** 

Predict which areas in Malawi are likely to be flooded following exceptionally heavy rainfall. Severe floods were caused in 2015 by heavy rain from December 2014 to March 2015, and then again in 2019 following Cyclone Ida’s passage through the area. The task is to predict which 1km squares of land were flooded in 2019, given knowledge of which squares flooded in 2015, plus location, elevation and land use data along with rainfall in the weeks before and during the flood.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import KFold
import lightgbm as lgbm

seed = 101

## Load data and preprocess

In [2]:
data = pd.read_csv("Train.csv")

# Construct train & test set
# Rename precipitation columns using week_1_precip, week_2_precip, etc so they are consistent across train & test set
precip_features_2019 = []
precip_features_2015 = []
for col in data.columns:
  if '2019' in col:
    precip_features_2019.append(col)
  elif 'precip 2014' in col:
    precip_features_2015.append(col)
  elif 'precip 2015' in col:
    precip_features_2015.append(col)

train = data[data.columns.difference(precip_features_2019)]
test = data[data.columns.difference(precip_features_2015)].drop('target_2015', axis = 1)

new_2015_cols = {}
for col, number in zip(precip_features_2015, range(1, len(precip_features_2015) + 1)):
  if 'precip' in col:
    new_2015_cols[col] = 'week_' + str(number) + '_precip'

new_2019_cols = {}
for col, number in zip(precip_features_2019, range(1, len(precip_features_2019) + 1)):
  if 'precip' in col:
    new_2019_cols[col] = 'week_' + str(number) + '_precip'

train.rename(columns = new_2015_cols, inplace = True)
test.rename(columns = new_2019_cols, inplace = True)

y = train['target_2015']
squareIDs = train['Square_ID']

train.drop(columns=['target_2015', 'Square_ID'], axis=1, inplace = True)
test.drop(columns=['Square_ID'], axis=1, inplace = True);

/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [3]:
train

,LC_Type1_mode,X,Y,elevation,week_1_precip,week_2_precip,week_3_precip,week_4_precip,week_5_precip,week_6_precip,...,week_8_precip,week_9_precip,week_10_precip,week_11_precip,week_12_precip,week_13_precip,week_14_precip,week_15_precip,week_16_precip,week_17_precip
0,9,34.26,-15.91,887.764222,0.000000,0.000000,0.000000,14.844025,14.552823,12.237766,...,30.127047,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.000000
1,9,34.26,-15.90,743.403912,0.000000,0.000000,0.000000,14.844025,14.552823,12.237766,...,30.127047,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.000000
2,9,34.26,-15.89,565.728343,0.000000,0.000000,0.000000,14.844025,14.552823,12.237766,...,30.127047,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.000000
3,10,34.26,-15.88,443.392774,0.000000,0.000000,0.000000,14.844025,14.552823,12.237766,...,30.127047,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.000000
4,10,34.26,-15.87,437.443428,0.000000,0.000000,0.000000,14.844025,14.552823,12.237766,...,30.127047,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16461,10,35.86,-15.44,635.675022,16.956563,31.155531,12.882013,8.810145,6.179829,9.863685,...,21.457507,105.275891,3.645338,18.531483,13.816063,23.728058,8.794998,9.369763,21.428131,2.493683
16462,10,35.86,-15.43,632.598892,16.956563,31.155531,12.882013,8.810145,6.179829,9.863685,...,21.457507,105.275891,3.645338,18.531483,13.816063,23.728058,8.794998,9.369763,21.428131,2.493683
16463,10,35.86,-15.42,632.450136,16.956563,31.155531,12.882013,8.810145,6.179829,9.863685,...,21.457507,105.275891,3.645338,18.531483,13.816063,23.728058,8.794998,9.369763,21.428131,2.493683
16464,10,35.86,-15.41,629.272733,16.956563,31.155531,12.882013,8.810145,6.179829,9.863685,...,21.457507,105.275891,3.645338,18.531483,13.816063,23.728058,8.794998,9.369763,21.428131,2.493683


In [4]:
test

,LC_Type1_mode,X,Y,elevation,week_1_precip,week_2_precip,week_3_precip,week_4_precip,week_5_precip,week_6_precip,...,week_8_precip,week_9_precip,week_10_precip,week_11_precip,week_12_precip,week_13_precip,week_14_precip,week_15_precip,week_16_precip,week_17_precip
0,9,34.26,-15.91,887.764222,12.992620,4.582856,35.037532,4.796012,28.083314,0.000000,...,18.264692,17.537486,0.896323,1.680000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,9,34.26,-15.90,743.403912,12.992620,4.582856,35.037532,4.796012,28.083314,0.000000,...,18.264692,17.537486,0.896323,1.680000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,9,34.26,-15.89,565.728343,12.992620,4.582856,35.037532,4.796012,28.083314,0.000000,...,18.264692,17.537486,0.896323,1.680000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,10,34.26,-15.88,443.392774,12.992620,4.582856,35.037532,4.796012,28.083314,0.000000,...,18.264692,17.537486,0.896323,1.680000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,10,34.26,-15.87,437.443428,12.992620,4.582856,35.037532,4.796012,28.083314,0.000000,...,18.264692,17.537486,0.896323,1.680000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16461,10,35.86,-15.44,635.675022,8.760326,5.177616,12.450319,17.289942,19.612179,10.909635,...,15.940852,24.828982,11.335339,30.984762,0.518269,5.770066,14.839779,4.928294,10.526186,18.746072
16462,10,35.86,-15.43,632.598892,8.760326,5.177616,12.450319,17.289942,19.612179,10.909635,...,15.940852,24.828982,11.335339,30.984762,0.518269,5.770066,14.839779,4.928294,10.526186,18.746072
16463,10,35.86,-15.42,632.450136,8.760326,5.177616,12.450319,17.289942,19.612179,10.909635,...,15.940852,24.828982,11.335339,30.984762,0.518269,5.770066,14.839779,4.928294,10.526186,18.746072
16464,10,35.86,-15.41,629.272733,8.760326,5.177616,12.450319,17.289942,19.612179,10.909635,...,15.940852,24.828982,11.335339,30.984762,0.518269,5.770066,14.839779,4.928294,10.526186,18.746072


## Initial Model

We start with an initial LightGBM model with default parameters.

In [5]:
# Define training method with 10-fold CV
# Predict on test set during each fold and average results

def train_model_10_fold(model, X, y, X_test):
    n_split=10
    kf = KFold(n_splits=10, shuffle=True, random_state=seed)
    rmses = []

    for t, v in kf.split(X, y):
        X_train, X_val, y_train, y_val = X.iloc[t], X.iloc[v], y.iloc[t], y.iloc[v]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        rmse = sqrt(mean_squared_error(y_val, y_pred))
        print(rmse)
        rmses.append(rmse)

    mean_rmse = np.mean(rmses)
    print('Mean RMSE:', mean_rmse)
    
    return mean_rmse

def train_model_full(model, X, y, X_test):
    model.fit(X, y)
    y_pred = model.predict(X)
    rmse = sqrt(mean_squared_error(y, y_pred))
    print('RMSE:', rmse)
    
    test_preds = model.predict(X_test)
    test_preds = [0 if y < 0.00 else y for y in test_preds] # Round predictions < 0 to 0
    return model, test_preds

def prepare_submission(squareIDs, test_preds):
    submission = pd.DataFrame({'Square_ID': squareIDs, 'target_2019': test_preds})
    submission.to_csv('submission.csv', index = False)

In [6]:
# Use LightGBM with default settings
model = lgbm.LGBMRegressor(
    n_jobs=5,
    random_state = seed,
)

train_model_10_fold(model, train, y, test);

0.10197202376360796
0.11029795461111432
0.09205689381840161
0.1002655414758398
0.09942152293492647
0.09268377998148912
0.1080242680867244
0.10192952893879487
0.08493402611006517
0.09529799010527132
Mean RMSE: 0.09868835298262349


In [7]:
trained_model, test_preds = train_model_full(model, train, y, test)

prepare_submission(squareIDs, test_preds)

RMSE: 0.08440325373187604


Leaderboard: 0.144713679106222

## Grid Search

We performed grid search over the main LightGBM parameters to find the optimum parameter values, namely the no. of estimators, no. of leaves and maximum depth. A wider range of values was used initially, before narrowing the range of values using smaller increments to determine the best hyperparameter values. We present the final grid search performed below:

In [8]:
n_estimators = [40,45,50,55,60,65,70]
num_leaves = [14,16,18,20,22,24]
max_depths = [4,6,8,10,12,14]

best_e = 0
best_l = 0
best_d = 0
best_rmse = 1

for e in n_estimators:
    for l in num_leaves:
        for d in max_depths:
            model = lgbm.LGBMRegressor(
                n_jobs=5,
                random_state=seed,
                n_estimators=e,
                num_leaves=l,
                max_depth=d,
            )
            
            print('e: {}, l: {}, d: {}'.format(e,l,d))
            
            rmse = train_model_10_fold(model, train, y, test)
            
            if rmse < best_rmse:
                best_e = e
                best_l = l
                best_d = d
                best_rmse = rmse

print('best e: {}, best l: {}, best d: {}'.format(best_e,best_l,best_d))

e: 40, l: 14, d: 4
0.13119214608762123
0.13170496848104785
0.1168182362182177
0.121934648821674
0.12720379096530718
0.1203656598632716
0.12831360522979643
0.12288206393131486
0.11360398131057123
0.12076954354490545
Mean RMSE: 0.12347886444537273
e: 40, l: 14, d: 6
0.12597874353720642
0.12629251242303038
0.1114226044989792
0.11877023586481589
0.12108604749973778
0.11408257371331333
0.12397805421181647
0.11913532801485419
0.10779765269367543
0.1137988997037059
Mean RMSE: 0.1182342652161135
e: 40, l: 14, d: 8
0.12398349084270438
0.12536398834781098
0.10822455997566875
0.11633989193658895
0.11985322173899132
0.11174839178048708
0.12269193661945028
0.11780426179320419
0.10562117868931485
0.11252839103871413
Mean RMSE: 0.11641593127629349
e: 40, l: 14, d: 10
0.1230901646238914
0.1258521687511706
0.10733423808485656
0.11697504389080347
0.11908419982982017
0.11181208332580603
0.12246610885342098
0.11771774375024599
0.10493602462503933
0.11210813374909805
Mean RMSE: 0.11613759094841528
e: 40, l

In [8]:
# Use LightGBM with best parameters found empirically
model = lgbm.LGBMRegressor(
    n_jobs=5,
    random_state = seed,
    n_estimators=70,
    num_leaves=24,
    max_depth=14,
)

train_model_10_fold(model, train, y, test);

0.1069889877241181
0.1128382690074341
0.09516792544319362
0.10384758105488412
0.1056712199508572
0.09806257758580603
0.1132746248637173
0.10708730559513707
0.090231992479189
0.10104344625629644
Mean RMSE: 0.1034213929960633


In [9]:
trained_model, test_preds = train_model_full(model, train, y, test)

prepare_submission(squareIDs, test_preds)

RMSE: 0.09211439650709176


Leaderboard: 0.133621777157139

We seem to get the lowest RMSE for 10-fold cross-validation when n_estimators, num_leaves and max_depth are at its maximum of the given range. There is only a slight improvement over the baseline.

Instead, we tried to obtain the best parameters through a few submissions, using a combination of parameter values in the ranges we defined earlier. Surprisingly, we were able to obtain better results, using the following parameters:

num_leaves = 50, num_leaves = 20, max_depth = 10

In [10]:
# Use LightGBM with best parameters found through submissions to the leaderboard
model = lgbm.LGBMRegressor(
    n_jobs=5,
    random_state = seed,
    n_estimators=50,
    num_leaves=20,
    max_depth=10,
)

train_model_10_fold(model, train, y, test);

0.11291592425416767
0.11791088393947409
0.0999399894339169
0.10842315262084941
0.11068840843741938
0.10321564030635443
0.1175689986410091
0.11235622706510058
0.09566378263935114
0.10298412083867653
Mean RMSE: 0.10816671281763193


In [11]:
trained_model, test_preds = train_model_full(model, train, y, test)

prepare_submission(squareIDs, test_preds)

RMSE: 0.10007585899646386


Leaderboard: 0.12008185182687

Analysis: The best parameters obtained by grid search has overfitted to the training data. This could be due to the fact that this is a one-shot learning problem -- we are trying to predict flooding on 2019 rainfall by training on 2015 rainfall data. However, by performing a manual grid search on our hyperparameters through a few submissions, we are able to improve the performance of our model by a large margin.

## Normalising

Attempt to see if normalising data helps improve performance.

In [12]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(train.values)

train_normalised = scaler.transform(train.values)
test_normalised = scaler.transform(test.values)

In [13]:
train_normalised

array([[4.66666667e-01, 0.00000000e+00, 5.10489510e-01, ...,
        3.28426728e-01, 9.38153879e-04, 0.00000000e+00],
       [4.66666667e-01, 0.00000000e+00, 5.17482517e-01, ...,
        3.28426728e-01, 9.38153879e-04, 0.00000000e+00],
       [4.66666667e-01, 0.00000000e+00, 5.24475524e-01, ...,
        3.28426728e-01, 9.38153879e-04, 0.00000000e+00],
       ...,
       [5.33333333e-01, 1.00000000e+00, 8.53146853e-01, ...,
        8.73478941e-02, 8.09012159e-01, 5.03436539e-01],
       [5.33333333e-01, 1.00000000e+00, 8.60139860e-01, ...,
        8.73478941e-02, 8.09012159e-01, 5.03436539e-01],
       [5.33333333e-01, 1.00000000e+00, 8.67132867e-01, ...,
        8.73478941e-02, 8.09012159e-01, 5.03436539e-01]])

In [14]:
def train_model_10_fold_numpy(model, X, y, X_test):
    n_split=10
    kf = KFold(n_splits=10, shuffle=True, random_state=seed)
    rmses = []

    for t, v in kf.split(X, y):
        X_train, X_val, y_train, y_val = X[t], X[v], y[t], y[v]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        rmse = sqrt(mean_squared_error(y_val, y_pred))
        print(rmse)
        rmses.append(rmse)

    mean_rmse = np.mean(rmses)
    print('Mean RMSE:', mean_rmse)
    
    return mean_rmse

In [15]:
# Use LightGBM with best parameters found empirically
model = lgbm.LGBMRegressor(
    n_jobs=5,
    random_state = seed,
    n_estimators=50,
    num_leaves=20,
    max_depth=10,
)

train_model_10_fold_numpy(model, train_normalised, y, test_normalised);

0.11431158553358108
0.11777061522395176
0.10050658879556246
0.10889873803769715
0.1093600184310815
0.10262548027217755
0.1175689986410091
0.11154618763685091
0.09669807982726589
0.10465940722289772
Mean RMSE: 0.1083945699622075


In [16]:
trained_model, test_preds = train_model_full(model, train_normalised, y, test_normalised)

prepare_submission(squareIDs, test_preds)

RMSE: 0.10049712464483941


Leaderboard: 0.124432508250531

Analysis: Normalising did not seem to result in any improvement on the model, which can be due to the fact that the values in our data are not extreme.

## Encoding

We didn't perform any encoding as we only have 1 categorical column (LC_Type1_mode) and LGBM is able to perform necessary encoding for categorical columns, as mentioned in their documentation.

## Feature Engineering

We attempt to engineer features in the following ways:

a) Adding 2 level interactions between features

In [17]:
from itertools import combinations
import copy

train_inter = copy.deepcopy(train)
test_inter = copy.deepcopy(test)
org_train_inter = copy.deepcopy(train_inter)

for c1,c2 in combinations(org_train_inter, 2):
    name = "{}_{}".format(c1,c2)
    train_inter[name] = train_inter[c1] * train_inter[c2]
    test_inter[name] = test_inter[c1] * test_inter[c2]

In [18]:
train_inter

,LC_Type1_mode,X,Y,elevation,week_1_precip,week_2_precip,week_3_precip,week_4_precip,week_5_precip,week_6_precip,...,week_13_precip_week_14_precip,week_13_precip_week_15_precip,week_13_precip_week_16_precip,week_13_precip_week_17_precip,week_14_precip_week_15_precip,week_14_precip_week_16_precip,week_14_precip_week_17_precip,week_15_precip_week_16_precip,week_15_precip_week_17_precip,week_16_precip_week_17_precip
0,9,34.26,-15.91,887.764222,0.000000,0.000000,0.000000,14.844025,14.552823,12.237766,...,7.885179,136.268144,27.029885,0.000000,16.059060,3.185444,0.000000,55.049421,0.000000,0.000000
1,9,34.26,-15.90,743.403912,0.000000,0.000000,0.000000,14.844025,14.552823,12.237766,...,7.885179,136.268144,27.029885,0.000000,16.059060,3.185444,0.000000,55.049421,0.000000,0.000000
2,9,34.26,-15.89,565.728343,0.000000,0.000000,0.000000,14.844025,14.552823,12.237766,...,7.885179,136.268144,27.029885,0.000000,16.059060,3.185444,0.000000,55.049421,0.000000,0.000000
3,10,34.26,-15.88,443.392774,0.000000,0.000000,0.000000,14.844025,14.552823,12.237766,...,7.885179,136.268144,27.029885,0.000000,16.059060,3.185444,0.000000,55.049421,0.000000,0.000000
4,10,34.26,-15.87,437.443428,0.000000,0.000000,0.000000,14.844025,14.552823,12.237766,...,7.885179,136.268144,27.029885,0.000000,16.059060,3.185444,0.000000,55.049421,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16461,10,35.86,-15.44,635.675022,16.956563,31.155531,12.882013,8.810145,6.179829,9.863685,...,208.688217,222.326282,508.447931,59.170255,82.407045,188.460362,21.931936,200.776508,23.365219,53.434965
16462,10,35.86,-15.43,632.598892,16.956563,31.155531,12.882013,8.810145,6.179829,9.863685,...,208.688217,222.326282,508.447931,59.170255,82.407045,188.460362,21.931936,200.776508,23.365219,53.434965
16463,10,35.86,-15.42,632.450136,16.956563,31.155531,12.882013,8.810145,6.179829,9.863685,...,208.688217,222.326282,508.447931,59.170255,82.407045,188.460362,21.931936,200.776508,23.365219,53.434965
16464,10,35.86,-15.41,629.272733,16.956563,31.155531,12.882013,8.810145,6.179829,9.863685,...,208.688217,222.326282,508.447931,59.170255,82.407045,188.460362,21.931936,200.776508,23.365219,53.434965


In [19]:
# Use LightGBM with best parameters found empirically
model = lgbm.LGBMRegressor(
    n_jobs=5,
    random_state = seed,
    n_estimators=50,
    num_leaves=20,
    max_depth=10,
)

train_model_10_fold(model, train_inter, y, test_inter);

0.11134109699134229
0.11608487758814345
0.09867763024355954
0.10908003666280919
0.10965240918628667
0.10122868020867624
0.11596861023930526
0.10483037990275021
0.09622627939573274
0.10220494977613452
Mean RMSE: 0.106529495019474


In [20]:
trained_model, test_preds = train_model_full(model, train_inter, y, test_inter)

prepare_submission(squareIDs, test_preds)

RMSE: 0.09715844104845114


Leaderboard: 0.291563137629413

b) 2 level interactions between X, Y and Elevation only as they have the highest feature importance

In [21]:
train_inter = copy.deepcopy(train)
test_inter = copy.deepcopy(test)
org_train_inter = copy.deepcopy(train_inter[['X','Y','elevation']])

for c1,c2 in combinations(org_train_inter, 2):
    name = "{}_{}".format(c1,c2)
    train_inter[name] = train_inter[c1] * train_inter[c2]
    test_inter[name] = test_inter[c1] * test_inter[c2]

In [22]:
train_inter

,LC_Type1_mode,X,Y,elevation,week_1_precip,week_2_precip,week_3_precip,week_4_precip,week_5_precip,week_6_precip,...,week_11_precip,week_12_precip,week_13_precip,week_14_precip,week_15_precip,week_16_precip,week_17_precip,X_Y,X_elevation,Y_elevation
0,9,34.26,-15.91,887.764222,0.000000,0.000000,0.000000,14.844025,14.552823,12.237766,...,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.000000,-545.0766,30414.802245,-14124.328772
1,9,34.26,-15.90,743.403912,0.000000,0.000000,0.000000,14.844025,14.552823,12.237766,...,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.000000,-544.7340,25469.018021,-11820.122199
2,9,34.26,-15.89,565.728343,0.000000,0.000000,0.000000,14.844025,14.552823,12.237766,...,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.000000,-544.3914,19381.853034,-8989.423372
3,10,34.26,-15.88,443.392774,0.000000,0.000000,0.000000,14.844025,14.552823,12.237766,...,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.000000,-544.0488,15190.636421,-7041.077244
4,10,34.26,-15.87,437.443428,0.000000,0.000000,0.000000,14.844025,14.552823,12.237766,...,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.000000,-543.7062,14986.811859,-6942.227210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16461,10,35.86,-15.44,635.675022,16.956563,31.155531,12.882013,8.810145,6.179829,9.863685,...,18.531483,13.816063,23.728058,8.794998,9.369763,21.428131,2.493683,-553.6784,22795.306294,-9814.822342
16462,10,35.86,-15.43,632.598892,16.956563,31.155531,12.882013,8.810145,6.179829,9.863685,...,18.531483,13.816063,23.728058,8.794998,9.369763,21.428131,2.493683,-553.3198,22684.996275,-9761.000907
16463,10,35.86,-15.42,632.450136,16.956563,31.155531,12.882013,8.810145,6.179829,9.863685,...,18.531483,13.816063,23.728058,8.794998,9.369763,21.428131,2.493683,-552.9612,22679.661879,-9752.381098
16464,10,35.86,-15.41,629.272733,16.956563,31.155531,12.882013,8.810145,6.179829,9.863685,...,18.531483,13.816063,23.728058,8.794998,9.369763,21.428131,2.493683,-552.6026,22565.720206,-9697.092816


In [23]:
# Use LightGBM with best parameters found empirically
model = lgbm.LGBMRegressor(
    n_jobs=5,
    random_state = seed,
    n_estimators=50,
    num_leaves=20,
    max_depth=10,
)

train_model_10_fold(model, train_inter, y, test_inter);

0.11303751968679702
0.1151637511277281
0.09772270933786494
0.10942106134689666
0.10670979758145137
0.10084302938829541
0.11165351181168467
0.10820924587542528
0.09465725763140109
0.10391194875873944
Mean RMSE: 0.10613298325462839


In [24]:
trained_model, test_preds = train_model_full(model, train_inter, y, test_inter)

prepare_submission(squareIDs, test_preds)

RMSE: 0.09814211169997239


Leaderboard: 0.123171563811687

c) Making a second copy of the dataset with no rain and no flooding to "tell" the model that flooding only happens when it rains

In [25]:
# Make the extra "no rain" dataset
train2 = train.copy()
for col in train2.columns:
      if 'week' in col:
            train2[col].values[:] = 0
            print(col)
y2 = y.copy()
y2.values[:] = 0
train_dbl = pd.concat([train,train2])
y_dbl = pd.concat([y,y2])

week_1_precip
week_2_precip
week_3_precip
week_4_precip
week_5_precip
week_6_precip
week_7_precip
week_8_precip
week_9_precip
week_10_precip
week_11_precip
week_12_precip
week_13_precip
week_14_precip
week_15_precip
week_16_precip
week_17_precip


In [26]:
# Use LightGBM with best parameters found empirically
model = lgbm.LGBMRegressor(
    n_jobs=5,
    random_state = seed,
    n_estimators=50,
    num_leaves=20,
    max_depth=10,
)
train_model_10_fold(model, train_dbl, y_dbl, test);

0.08262334682963607
0.0772169894928063
0.07237287278941108
0.0742951785821473
0.08375156523491643
0.0812487159300801
0.07887405475409524
0.07442602596028203
0.07768846056587635
0.07149942488665789
Mean RMSE: 0.07739966350259089


In [27]:
trained_model, test_preds = train_model_full(model, train_dbl, y_dbl, test)

prepare_submission(squareIDs, test_preds)

RMSE: 0.07155590324796147


Leaderboard: 0.127468864109669

d) Slope and valley calculation - Hypothesis is that slope will have additional explanatory power over and above elevation
and that valley/hollow areas may be more flood-prone

In [28]:
# Slope and valley/hollow calculation
def add_slope(train):
    tsize = len(train)
    slopeL = np.zeros((tsize,1))
    slopeR = np.zeros((tsize,1))
    slopeA = np.zeros((tsize,1))
    slopeB = np.zeros((tsize,1))
    valleyLR = np.zeros((tsize,1))
    valleyAB = np.zeros((tsize,1))
    hollow = np.zeros((tsize,1))

    for i, row in enumerate(train.itertuples()):
        x0 = row.X
        y0 = row.Y
        above = train[(train['X'] == x0) & (abs(train['Y']-y0-0.01) < 1e-6)]
        below = train[(train['X'] == x0) & (abs(train['Y']-y0+0.01) < 1e-6)]
        left =  train[(train['Y'] == y0) & (abs(train['X']-x0-0.01) < 1e-6)]
        right = train[(train['Y'] == y0) & (abs(train['X']-x0+0.01) < 1e-6)]
        slopeL[i] = (left.elevation - row.elevation).values.item() if len(left) == 1 else 0
        slopeR[i] = (right.elevation - row.elevation).values.item() if len(right) == 1 else 0
        slopeA[i] = (above.elevation - row.elevation).values.item() if len(above) == 1 else 0
        slopeB[i] = (below.elevation - row.elevation).values.item() if len(below) == 1 else 0
        valleyLR[i] = (slopeL[i] > 10 and slopeR[i]> 10)
        valleyAB[i] = (slopeA[i] > 10 and slopeB[i]> 10)
        hollow[i] = valleyLR[i] * valleyAB[i]
    
    df = pd.DataFrame(np.hstack((slopeL,slopeR,slopeA,slopeB
                                 ,valleyLR,valleyAB,hollow
                                )),  
                                  columns=['slopeL','slopeR','slopeA','slopeB','valleyLR','valleyAB','hollow'])
    train_enh = pd.concat([train,df],axis=1)
    return train_enh

In [29]:
# Engineer the slope/valley data into another dataset
train_slope = add_slope(train)
test_slope = add_slope(test)

In [30]:
# Use LightGBM with best parameters found empirically
model = lgbm.LGBMRegressor(
    n_jobs=5,
    random_state = seed,
    n_estimators=50,
    num_leaves=20,
    max_depth=10,
)
train_model_10_fold(model, train_slope, y, test_slope);

0.10903968829278661
0.11354334053616273
0.09947235888823983
0.10393007807368077
0.10720562631570886
0.1029943799429969
0.11312052322618833
0.10909778812247749
0.09737114762437736
0.0997623546418481
Mean RMSE: 0.1055537285664467


In [31]:
trained_model, test_preds = train_model_full(model, train_slope, y, test_slope)

prepare_submission(squareIDs, test_preds)

RMSE: 0.09457539625823923


Leaderboard: 0.135622372723607


Analysis: Feature engineering did not seem to improve our model at all. This can be due to LightGBM being able to capture feature interactions innately.

## Stacking

We attempt to stack with other models. Various boosting models and random forest have been tried in different combinations, using RidgeCV as our final estimator.

In [32]:
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import RidgeCV

estimators = [
    ('Random Forest', RandomForestRegressor(random_state=seed)),
    ('AdaBoost', AdaBoostRegressor(random_state=seed)),
    ('Gradient Boosting', GradientBoostingRegressor(random_state=seed)),
#     ('XGBoost', XGBRegressor(objective ='reg:squarederror')),
#     ('CatBoost', CatBoostRegressor(logging_level='Silent')),
    ('LGBM', lgbm.LGBMRegressor(
        n_estimators=50,
        n_jobs=5,
        random_state = seed,
        num_leaves=20,
        max_depth=10,
    ))
]

In [33]:
# Use LightGBM with best parameters found empirically
model = StackingRegressor(
    estimators=estimators, 
    final_estimator=RidgeCV()
)

train_model_10_fold(model, train, y, test);

0.1137159181499147
0.11767804576541116
0.10819848958321614
0.10851973740507459
0.11529362227188816
0.11041797226869958
0.11318691957107826
0.11081712493384362
0.10634783548853362
0.10314713708826441
Mean RMSE: 0.11073228025259244


In [34]:
trained_model, test_preds = train_model_full(model, train, y, test)

prepare_submission(squareIDs, test_preds)

RMSE: 0.10314005583045359


Leaderboard: 0.128612348683132

Analysis: We capture only one out of the many stacking combinations that we have tried. However, none of them seemed to outperform the single LightGBM model with optimal hyperparameters that we started out with. Surprisingly, when we trained the stacked model below with just the same LightGBM model, we were able to outperform that.

In [35]:
estimators = [
    ('LGBM', lgbm.LGBMRegressor(
        n_estimators=50,
        n_jobs=5,
        random_state = seed,
        num_leaves=20,
        max_depth=10,
    ))
]

In [36]:
# Use LightGBM with best parameters, and a RidgeCV on top
model = StackingRegressor(
    estimators=estimators, 
    final_estimator=RidgeCV()
)

train_model_10_fold(model, train, y, test);

0.1210943857247277
0.12527753961238752
0.10954376388453883
0.11436401768479384
0.12024890685895279
0.11053541701219928
0.12407115821050896
0.11546410428196853
0.11261768668987866
0.11136884077457795
Mean RMSE: 0.11645858207345339


In [37]:
trained_model, test_preds = train_model_full(model, train, y, test)

prepare_submission(squareIDs, test_preds)

RMSE: 0.1101708611129125


Leaderboard: 0.10838585681909

Analysis: From the above experiments, it seemed like any additional model types used did not help to improve the performance of the stacked model. One explanation could be that LightGBM was able to generalise best out of all the models and adding additional model types only resulted in overfitting to the data.

## Adding data

We attempt to add more data from Google Earth Engine to see if we can improve performance:

The following datasets are added:

Landform classes: https://developers.google.com/earth-engine/datasets/catalog/CSP_ERGo_1_0_Global_ALOS_landforms

Soil moisture: https://developers.google.com/earth-engine/datasets/catalog/NASA_USDA_HSL_soil_moisture?hl=en

Soil density: https://developers.google.com/earth-engine/datasets/catalog/OpenLandMap_SOL_SOL_BULKDENS-FINEEARTH_USDA-4A1H_M_v02

**Land cover type**: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MCD12Q1

Each dataset was obtained from Google Earth Engine (code to extract can be found in EarthEngineCode.js) and added to the training set separately before training and testing if the additional data helps to improve the performance of the model. However, the only data that we have found to have helped is the additional land cover type data, which is documented below:

In [38]:
dataWithLC = pd.read_csv("TrainWithLC.csv")

# Construct train & test data
# Replace precipitation columns using week_1_precip, week_2_precip, etc
precip_features_2019 = []
precip_features_2015 = []
for col in data.columns:
  if '2019' in col:
    precip_features_2019.append(col)
  elif 'precip 2014' in col:
    precip_features_2015.append(col)
  elif 'precip 2015' in col:
    precip_features_2015.append(col)

trainWithLC = dataWithLC[dataWithLC.columns.difference(precip_features_2019)]
testWithLC = dataWithLC[dataWithLC.columns.difference(precip_features_2015)].drop('target_2015', axis = 1)

new_2015_cols = {}
for col, number in zip(precip_features_2015, range(1, len(precip_features_2015) + 1)):
  if 'precip' in col:
    new_2015_cols[col] = 'week_' + str(number) + '_precip'

new_2019_cols = {}
for col, number in zip(precip_features_2019, range(1, len(precip_features_2019) + 1)):
  if 'precip' in col:
    new_2019_cols[col] = 'week_' + str(number) + '_precip'

trainWithLC.rename(columns = new_2015_cols, inplace = True)
testWithLC.rename(columns = new_2019_cols, inplace = True)

trainWithLC.drop(columns=['target_2015', 'Square_ID', 'LC_Type1'], axis=1, inplace = True)
testWithLC.drop(columns=['Square_ID', 'LC_Type1'], axis=1, inplace = True);

/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [39]:
trainWithLC

,LC_Prop1,LC_Prop1_Assessment,LC_Prop2,LC_Prop2_Assessment,LC_Prop3,LC_Prop3_Assessment,LC_Type1_mode,LC_Type2,LC_Type3,LC_Type4,...,week_8_precip,week_9_precip,week_10_precip,week_11_precip,week_12_precip,week_13_precip,week_14_precip,week_15_precip,week_16_precip,week_17_precip
0,22.0,98.0,20,98.0,20,98.0,9,9.0,4.0,4.0,...,30.127047,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.000000
1,22.0,98.0,20,98.0,20,98.0,9,9.0,4.0,4.0,...,30.127047,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.000000
2,22.0,98.0,20,98.0,20,98.0,9,9.0,4.0,4.0,...,30.127047,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.000000
3,31.0,98.0,30,96.0,30,98.0,10,10.0,1.0,6.0,...,30.127047,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.000000
4,31.0,98.0,30,85.0,30,98.0,10,10.0,1.0,6.0,...,30.127047,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16461,31.0,99.0,30,99.0,30,99.0,10,10.0,1.0,6.0,...,21.457507,105.275891,3.645338,18.531483,13.816063,23.728058,8.794998,9.369763,21.428131,2.493683
16462,31.0,99.0,30,99.0,30,99.0,10,10.0,1.0,6.0,...,21.457507,105.275891,3.645338,18.531483,13.816063,23.728058,8.794998,9.369763,21.428131,2.493683
16463,31.0,99.0,30,99.0,30,99.0,10,10.0,1.0,6.0,...,21.457507,105.275891,3.645338,18.531483,13.816063,23.728058,8.794998,9.369763,21.428131,2.493683
16464,31.0,99.0,30,99.0,30,99.0,10,10.0,1.0,6.0,...,21.457507,105.275891,3.645338,18.531483,13.816063,23.728058,8.794998,9.369763,21.428131,2.493683


In [40]:
# Use LightGBM with best parameters, and a RidgeCV on top
model = StackingRegressor(
    estimators=[
        ('LGBM', lgbm.LGBMRegressor(
            n_estimators=50,
            n_jobs=5,
            random_state = seed,
            num_leaves=20,
            max_depth=10,
        ))
    ], 
    final_estimator=RidgeCV()
)

train_model_10_fold(model, trainWithLC, y, testWithLC);

0.12440941184281518
0.12358622275298091
0.10650647955069154
0.11655711828153123
0.11780356072778343
0.11277902974143832
0.12525093656061154
0.1142466321721379
0.1101639047965128
0.10730478454744916
Mean RMSE: 0.11586080809739518


In [41]:
trained_model, test_preds = train_model_full(model, trainWithLC, y, testWithLC)

prepare_submission(squareIDs, test_preds)

RMSE: 0.11051405948339048


Leaderboard: 0.105109848621334

Analysis: Adding data on Land Cover seemed to help improve the model's predictive capability by a small but significant margin.

## Multi-Layer Stacking

After trying out a single-layer stacking and adding data, we attempt to stack layers of LightGBM model.

In [42]:
l1_estimators = [
    ('LGBM', lgbm.LGBMRegressor(
        n_estimators=50,
        n_jobs=5,
        random_state=seed,
        num_leaves=20,
        max_depth=10,
    ))
]
l2_estimators = [
    ('LGBM', lgbm.LGBMRegressor(
        n_estimators=50,
        n_jobs=5,
        random_state=seed,
        num_leaves=20,
        max_depth=10,
    ))
]

In [43]:
# Use 2 layers of LightGBM with best parameters, with a RidgeCV on top
model = StackingRegressor(
    estimators=l1_estimators,
    final_estimator=StackingRegressor(
        estimators=l2_estimators,
        final_estimator=RidgeCV()
    )
)

train_model_10_fold(model, trainWithLC, y, testWithLC);

0.1440273640501402
0.14721268817171315
0.1272094066765182
0.13191931503257032
0.13212921652603063
0.13702912225851344
0.14424439948409007
0.12957101270688565
0.13721737648985088
0.132678230051418
Mean RMSE: 0.13632381314477307


In [44]:
trained_model, test_preds = train_model_full(model, trainWithLC, y, testWithLC)

prepare_submission(squareIDs, test_preds)

RMSE: 0.13190574065325608


Leaderboard: 0.109041435133235

Analysis: Stacking 2 layers of LightGBM models didn't seem to improve performance, but we have not yet finetuned the stacked model. Thus, we perform grid search on the hyperparameters of the 2nd layer.

In [13]:
n_estimators = [10,15,20,25,30]
num_leaves = [2,3,4,6,8,10]
max_depths = [2,3,4,6,8,10]

best_e = 0
best_l = 0
best_d = 0
best_rmse = 1

for e in n_estimators:
    for l in num_leaves:
        for d in max_depths:
            model = StackingRegressor(
                estimators=l1_estimators, 
                final_estimator=StackingRegressor(
                    estimators=[
                        ('LGBM', lgbm.LGBMRegressor(
                            n_estimators=e,
                            n_jobs=5,
                            random_state=seed,
                            num_leaves=l,
                            max_depth=d,
                        ))
                    ],
                    final_estimator=RidgeCV()
                )
            )
            
            print('e: {}, l: {}, d: {}'.format(e,l,d))
            
            rmse = train_model_10_fold(model, trainWithLC, y, testWithLC)
            
            if rmse < best_rmse:
                best_e = e
                best_l = l
                best_d = d
                best_rmse = rmse

print('best e: {}, best l: {}, best d: {}'.format(best_e,best_l,best_d))

e: 10, l: 2, d: 2
0.1432995990292804
0.12985385903964983
0.12208017129752126
0.12869063266559302
0.12254536385796257
0.12568352211485403
0.14107653525342556
0.1282805713326966
0.12189824454112066
0.11921562854688907
Mean RMSE: 0.12826241276789932
e: 10, l: 2, d: 3
0.1432995990292804
0.12985385903964983
0.12208017129752126
0.12869063266559302
0.12254536385796257
0.12568352211485403
0.14107653525342556
0.1282805713326966
0.12189824454112066
0.11921562854688907
Mean RMSE: 0.12826241276789932
e: 10, l: 2, d: 4
0.1432995990292804
0.12985385903964983
0.12208017129752126
0.12869063266559302
0.12254536385796257
0.12568352211485403
0.14107653525342556
0.1282805713326966
0.12189824454112066
0.11921562854688907
Mean RMSE: 0.12826241276789932
e: 10, l: 2, d: 6
0.1432995990292804
0.12985385903964983
0.12208017129752126
0.12869063266559302
0.12254536385796257
0.12568352211485403
0.14107653525342556
0.1282805713326966
0.12189824454112066
0.11921562854688907
Mean RMSE: 0.12826241276789932
e: 10, l: 2,

0.127812998011506
0.12958173955167873
0.1278113299551077
0.1437072316894204
0.12961435754575834
0.13232535396680112
0.12337680789069473
Mean RMSE: 0.13259733993097528
e: 10, l: 10, d: 8
0.14606594238798867
0.1454472688785486
0.12059814632116522
0.12812431406166935
0.1298997273218614
0.12780473167761194
0.14307478926370898
0.12989745104660572
0.13178559700355388
0.12415009750875625
Mean RMSE: 0.132684806547147
e: 10, l: 10, d: 10
0.14606594238798867
0.1454472688785486
0.12059814632116522
0.12812431406166935
0.1298997273218614
0.12780473167761194
0.14340491752248138
0.1299117166582492
0.13178559700355388
0.12415009750875625
Mean RMSE: 0.1327192459341886
e: 15, l: 2, d: 2
0.14207619398048416
0.1318564594887629
0.1177652644766954
0.12523157941853386
0.11726142380507985
0.12006688296296693
0.13804732789627847
0.12552080155765133
0.12172428052897633
0.11589720336564277
Mean RMSE: 0.1255447417481072
e: 15, l: 2, d: 3
0.14207619398048416
0.1318564594887629
0.1177652644766954
0.1252315794185338

0.12700864704956
0.12460314753907871
0.11788810595408959
Mean RMSE: 0.12724867741442186
e: 15, l: 10, d: 3
0.14598819905067334
0.14071959747860072
0.1189987471535749
0.12605277796623685
0.12629989712749148
0.12497997256599785
0.14106421165452768
0.13170842850565778
0.12907170956333697
0.12142358702730575
Mean RMSE: 0.13063071280934033
e: 15, l: 10, d: 4
0.1452796237882696
0.1442525384494766
0.11997372847622302
0.12809182956528017
0.12700942837000406
0.12680428580425082
0.141953235790922
0.12949487296754095
0.13378697229569272
0.12508410092411817
Mean RMSE: 0.13217306164317782
e: 15, l: 10, d: 6
0.14550339000705917
0.14439229833563336
0.11999156124488271
0.12808762095537402
0.1286829213829299
0.1265058331667501
0.14215874507153092
0.12895293852860062
0.13194535394076837
0.12479279727041753
Mean RMSE: 0.13210134599039466
e: 15, l: 10, d: 8
0.14590706345024113
0.14510686978989376
0.1206355368384901
0.12772987418959603
0.1291411484455469
0.1268877151638939
0.14308086329163944
0.12928201656

0.12342060163226089
Mean RMSE: 0.13148216953305158
e: 20, l: 8, d: 8
0.14491699043296882
0.14400803826368833
0.11895108056019163
0.12825950124848365
0.12767676788061325
0.12560456303107403
0.14184324451832728
0.12930861980431768
0.13190360275000992
0.12321821651062233
Mean RMSE: 0.1315690625000297
e: 20, l: 8, d: 10
0.14491699043296882
0.14400803826368833
0.11895108056019163
0.12825950124848365
0.12767676788061325
0.12560456303107403
0.14184324451832728
0.12930861980431768
0.13190360275000992
0.12321821651062233
Mean RMSE: 0.1315690625000297
e: 20, l: 10, d: 2
0.14298317916051434
0.13909550209806146
0.11719632716041181
0.1226508039525466
0.11973687221792147
0.12349701103664354
0.13727525947933142
0.12468230705678458
0.12489511439792174
0.11680784376697112
Mean RMSE: 0.12688202203271082
e: 20, l: 10, d: 3
0.1443959010547158
0.1396005031832316
0.11873041814368535
0.12621088772500413
0.12570274210636492
0.12401725967061068
0.140992066074604
0.1304286716558485
0.12930740827049703
0.1214112

0.1439242182122788
0.14046104258553382
0.11844035789670937
0.12529288304270533
0.12543461111372484
0.1236075870898196
0.14056106308125957
0.1294817024988702
0.12850147855081726
0.12133080812362328
Mean RMSE: 0.12970357521953418
e: 25, l: 8, d: 4
0.14402193985604952
0.14094119100162134
0.11853770459710057
0.1275404414516509
0.1259617372435486
0.12573110414018798
0.1406473401404951
0.12756280548864993
0.13101035808980327
0.12359654626292926
Mean RMSE: 0.13055511682720367
e: 25, l: 8, d: 6
0.14440511902968917
0.14351822171009782
0.11929601165349585
0.1280158832129764
0.12804764329627857
0.12571815417563512
0.14195815371657905
0.12899182956974137
0.13132478431078978
0.12412781260717651
Mean RMSE: 0.13154036132824598
e: 25, l: 8, d: 8
0.1445481811832369
0.14375234469187623
0.11914852978529177
0.1285282817188762
0.1280002047230499
0.12621268020005189
0.1416189261949278
0.12879359116139003
0.13178860554772912
0.1244524815832818
Mean RMSE: 0.13168438267897115
e: 25, l: 8, d: 10
0.1445481811832

0.1262240218698031
0.12438688813580465
0.1404634008292397
0.12873518561223896
0.13030524340563474
0.12040487055911073
Mean RMSE: 0.13013537072037612
e: 30, l: 6, d: 10
0.1445060650270634
0.14194180928034034
0.11784162265465789
0.1265445998298676
0.1262240218698031
0.12438688813580465
0.1404634008292397
0.12873518561223896
0.13030524340563474
0.12040487055911073
Mean RMSE: 0.13013537072037612
e: 30, l: 8, d: 2
0.14198500052194535
0.13889099436645638
0.11680372499296693
0.12317329240585213
0.1214392932140558
0.12213100119938612
0.1371533529400433
0.12680328248261097
0.12582338885401845
0.11693275481606087
Mean RMSE: 0.1271136085793396
e: 30, l: 8, d: 3
0.14366597497624317
0.14053829419305786
0.11911363715775812
0.12610646995352348
0.12634932082984715
0.1241294354713817
0.14086560874650123
0.1285753112532736
0.12836668607089188
0.12091921455411145
Mean RMSE: 0.12986299532065895
e: 30, l: 8, d: 4
0.1434540993927492
0.1409542687532484
0.11907566091812694
0.1272315876783345
0.125516248931051

We use the optimal 2nd layer hyperparameters found via grid search and run the model on the full training dataset below:

In [45]:
# Use 2 layers of LightGBM using optimal hyperparameters for the 2nd layer found using grid search
model = StackingRegressor(
    estimators=l1_estimators,
    final_estimator=StackingRegressor(
        estimators=[
            ('LGBM', lgbm.LGBMRegressor(
                n_estimators=20,
                n_jobs=5,
                random_state=seed,
                num_leaves=2,
                max_depth=2,
            ))
        ],
        final_estimator=RidgeCV()
    )
)

train_model_10_fold(model, trainWithLC, y, testWithLC);

0.14044751422848692
0.13207874736699016
0.1158901059308214
0.12253297297335834
0.11624693777025266
0.1185612914863463
0.13545375533233847
0.12604986333195592
0.11965666297696198
0.11309497408303804
Mean RMSE: 0.12400128254805501


In [46]:
trained_model, test_preds = train_model_full(model, trainWithLC, y, testWithLC)

prepare_submission(squareIDs, test_preds)

RMSE: 0.11874568832815086


Leaderboard: 0.100106431877455

We attempt to add a third layer, using the best hyperparameters we found so far for layers 1 and 2.

In [17]:
n_estimators = [10,15,20,25,30]
num_leaves = [2,3,4,6,8,10]
max_depths = [2,3,4,6,8,10]

best_e = 0
best_l = 0
best_d = 0
best_rmse = 1

l2_estimators = [
    ('LGBM', lgbm.LGBMRegressor(
        n_estimators=20,
        n_jobs=5,
        random_state=seed,
        num_leaves=2,
        max_depth=2,
    ))
]

for e in n_estimators:
    for l in num_leaves:
        for d in max_depths:
            model = StackingRegressor(
                estimators=l1_estimators, 
                final_estimator=StackingRegressor(
                    estimators=l2_estimators,
                    final_estimator=StackingRegressor(
                        estimators=[
                            ('LGBM', lgbm.LGBMRegressor(
                                n_estimators=e,
                                n_jobs=5,
                                random_state=seed,
                                num_leaves=l,
                                max_depth=d,
                            ))
                        ],
                        final_estimator=RidgeCV()
                    )
                )
            )
            
            print('e: {}, l: {}, d: {}'.format(e,l,d))
            
            rmse = train_model_10_fold(model, trainWithLC, y, testWithLC)
            
            if rmse < best_rmse:
                best_e = e
                best_l = l
                best_d = d
                best_rmse = rmse

print('best e: {}, best l: {}, best d: {}'.format(best_e,best_l,best_d))

e: 10, l: 2, d: 2
0.16669997511199788
0.1528715390369239
0.12829706682952358
0.13079893513927984
0.12433096955689495
0.13524081641856803
0.15325187030190335
0.15215024891052933
0.11994460728320504
0.12515555961897942
Mean RMSE: 0.13887415882078055
e: 10, l: 2, d: 3
0.16669997511199788
0.1528715390369239
0.12829706682952358
0.13079893513927984
0.12433096955689495
0.13524081641856803
0.15325187030190335
0.15215024891052933
0.11994460728320504
0.12515555961897942
Mean RMSE: 0.13887415882078055
e: 10, l: 2, d: 4
0.16669997511199788
0.1528715390369239
0.12829706682952358
0.13079893513927984
0.12433096955689495
0.13524081641856803
0.15325187030190335
0.15215024891052933
0.11994460728320504
0.12515555961897942
Mean RMSE: 0.13887415882078055
e: 10, l: 2, d: 6
0.16669997511199788
0.1528715390369239
0.12829706682952358
0.13079893513927984
0.12433096955689495
0.13524081641856803
0.15325187030190335
0.15215024891052933
0.11994460728320504
0.12515555961897942
Mean RMSE: 0.13887415882078055
e: 10, l

0.12363166240985873
0.13214931687381973
0.11953132038080617
0.126767916188404
0.14952052258998366
0.14227478809776004
0.13344968510463295
0.12764046925491615
Mean RMSE: 0.13510761098231477
e: 10, l: 10, d: 8
0.1611056174883764
0.1355199201079866
0.12353194436005344
0.13214919792650992
0.11976862044903776
0.12682845652556424
0.14954991898737496
0.14165590412256404
0.1333326423598345
0.12764046925491615
Mean RMSE: 0.1351082691582218
e: 10, l: 10, d: 10
0.16097461426064025
0.1355199201079866
0.12353194436005344
0.13214919792650992
0.11976862044903776
0.12682845652556424
0.14954991898737496
0.14165590412256404
0.1333326423598345
0.12764046925491615
Mean RMSE: 0.1350951688354482
e: 15, l: 2, d: 2
0.16638264200952219
0.1516767900010633
0.12801599709674463
0.1281641672813553
0.12157510615407076
0.12798604151322993
0.1518246167998538
0.14645616310635112
0.11659783006880882
0.12121457872295384
Mean RMSE: 0.1359893932753954
e: 15, l: 2, d: 3
0.16638264200952219
0.1516767900010633
0.1280159970967

0.14811457343309115
0.14491940299094414
0.1145905647751169
0.12076751126520621
Mean RMSE: 0.1341723854784929
e: 15, l: 10, d: 3
0.16088637008725165
0.14167949045785658
0.12170725099645723
0.12872748384209692
0.11926896910686485
0.12383721958539139
0.1467821281555941
0.14591776449578672
0.12884860204501083
0.1208906830261673
Mean RMSE: 0.13385459617984777
e: 15, l: 10, d: 4
0.1603520376917124
0.1386776201462654
0.1220116222826321
0.1313124340013228
0.12117804926092839
0.12153262140029687
0.1497276604435773
0.13841766073751824
0.1335903448328823
0.12315623251709683
Mean RMSE: 0.13399562833142326
e: 15, l: 10, d: 6
0.16049853006111417
0.13606990785956066
0.12267589112444743
0.13246296879107713
0.12009828575967353
0.12501493932949737
0.14974892944649895
0.14150244361255226
0.13277278161686282
0.127714741138948
Mean RMSE: 0.13485594187402325
e: 15, l: 10, d: 8
0.1609877273616955
0.13601883110366617
0.12265285302179522
0.13243341991796062
0.1201615660657336
0.125001263313701
0.14976307536098

0.16149086647922756
0.13821382088218598
0.12393757085085746
0.13158764507104595
0.11883811585935887
0.12413877358605298
0.1503024916752884
0.14211014846645245
0.12832532837786054
0.12716395610291306
Mean RMSE: 0.13461087173512432
e: 20, l: 8, d: 10
0.16149086647922756
0.13821382088218598
0.12393757085085746
0.13158764507104595
0.11883811585935887
0.12413877358605298
0.1503024916752884
0.14211014846645245
0.12832532837786054
0.12716395610291306
Mean RMSE: 0.13461087173512432
e: 20, l: 10, d: 2
0.16502225209761154
0.1484576459307772
0.12789328097346656
0.12401452840574756
0.11617669837370517
0.11999395627662596
0.1473904854437887
0.14329560926737517
0.11293359208243126
0.11781238689751301
Mean RMSE: 0.13229904357490418
e: 20, l: 10, d: 3
0.16035083941336492
0.13721329169877836
0.12140894255414561
0.12923022835183845
0.11982597497923549
0.1243708597122854
0.1474462313378672
0.1447823860950106
0.1297951053857153
0.1214876258073119
Mean RMSE: 0.13359114853355533
e: 20, l: 10, d: 4
0.1609402

0.12036610318641085
0.12401810614985509
0.14809300881371548
0.1433657785378808
0.130539659629671
0.12259167460925896
Mean RMSE: 0.13370194589031875
e: 25, l: 8, d: 4
0.1610334088895526
0.13497750884723567
0.12244799946914413
0.13169221940295633
0.1194359882297842
0.12260636720119443
0.15010553791632258
0.14151429037949517
0.13042694576315864
0.12574627856192552
Mean RMSE: 0.13399865446607692
e: 25, l: 8, d: 6
0.1616420491828414
0.13651955954711
0.12387133402343256
0.13174376394708232
0.11892023517093625
0.12271865351016634
0.15060481472553583
0.14131450212267635
0.12981712182138586
0.12699438237404156
Mean RMSE: 0.13441464164252084
e: 25, l: 8, d: 8
0.16154945275574314
0.13652667067253915
0.12399918201674134
0.13174376394708232
0.11892023517093625
0.1227245067647452
0.15060481472553583
0.14131450212267635
0.12981712182138586
0.12704661002507195
Mean RMSE: 0.13442468600224575
e: 25, l: 8, d: 10
0.16154945275574314
0.13652667067253915
0.12399918201674134
0.13174376394708232
0.11892023517

0.12626888742525838
Mean RMSE: 0.13408329518075995
e: 30, l: 6, d: 10
0.1615797258349423
0.13726829178697839
0.12358537890172897
0.1300257079625724
0.11811123708565129
0.12396600956995116
0.15145008651952543
0.14234897442451805
0.12622865229647298
0.12626888742525838
Mean RMSE: 0.13408329518075995
e: 30, l: 8, d: 2
0.16228542448867575
0.14232948368405415
0.12482847829831929
0.12488806117762288
0.1164766649922166
0.1205414469206904
0.14834726978146776
0.14287886617281711
0.1152345269679621
0.11952767961190829
Mean RMSE: 0.13173379020957343
e: 30, l: 8, d: 3
0.16003937696868645
0.13485945591850507
0.1213968111309512
0.13089844981848953
0.12088049759899187
0.12313016939399414
0.1485149407730822
0.1424331274384278
0.12954070156057118
0.12266439995732986
Mean RMSE: 0.13343579305590292
e: 30, l: 8, d: 4
0.16127473111208068
0.13462109938043107
0.12281242155381639
0.13205028547893144
0.11963078574045657
0.12284819146722702
0.15026606237938314
0.14112690349418638
0.13069025478340118
0.125653894

In [55]:
l1_estimators = [
    ('LGBM', lgbm.LGBMRegressor(
        n_estimators=50,
        n_jobs=5,
        random_state=seed,
        num_leaves=20,
        max_depth=10,
    ))
]

l2_estimators = [
    ('LGBM', lgbm.LGBMRegressor(
        n_estimators=20,
        n_jobs=5,
        random_state=seed,
        num_leaves=2,
        max_depth=2,
    ))
]

# Use 3 layers of LightGBM using optimal hyperparameters found for all 3 layers
model = StackingRegressor(
    estimators=l1_estimators,
    final_estimator=StackingRegressor(
        estimators=l2_estimators,
        final_estimator=StackingRegressor(
            estimators=[
                ('LGBM', lgbm.LGBMRegressor(
                    n_estimators=25,
                    n_jobs=5,
                    random_state=seed,
                    num_leaves=4,
                    max_depth=2,
                ))
            ],
            final_estimator=RidgeCV()
        )
    )
)

train_model_10_fold(model, trainWithLC, y, testWithLC);

0.16331685756386363
0.14353559098685498
0.12632402753879965
0.12447388874244804
0.1164075244185003
0.11984115806650293
0.1471833629138436
0.14256884688555999
0.11457087475829768
0.11865461189421579
Mean RMSE: 0.13168767437688866


In [56]:
trained_model, test_preds = train_model_full(model, trainWithLC, y, testWithLC)

prepare_submission(squareIDs, test_preds)

RMSE: 0.12820859524183179


**Final Public Leaderboard: 0.0974780194397044**

Analysis: Stacking has helped to improve our model by a significant margin, up to 3 layers.